# JSON 형태의 행정 문서 대상 기계독해 데이터 정제하기
- 출처: AI허브 -> 한국어/텍스트 조회 -> "행정 문서 대상 기계독해 데이터"

In [1]:
!pip install datasets==2.21.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86

In [2]:
import requests
import json
import time
import pandas as pd
from datasets import Dataset
import huggingface_hub

In [3]:
def load_training_data():
    try:
        start_time = time.time() # 시작 시간

        alpaca_data = [] # 정제된 데이터가 담길 배열
        column_list = ['instruction', 'input', 'output'] # column

        # 다운로드 URL 정의
        url = "https://drive.google.com/uc?export=download&id=1JehPjKhyw9NlVdoKBv7ZtxZLqaEjN7H9"
        response = requests.get(url)

        original_data = json.loads(response.text)

        original_data_key = list(original_data.keys()) # ['Dataset', 'data']
        datas = original_data[original_data_key[1]]

        # 모든 data 순회
        for eachData in datas:
            eachParagraphs = eachData["paragraphs"][0]
            data_key = eachParagraphs.keys() # ['context', 'qas', 'context_id']
            qasData = eachParagraphs["qas"]
            # Q&A 질의문 순회
            for eachData in qasData:
                instrunction = eachData["question"] # 질문
                ansData = eachData["answers"]
                # 제시문1 + "\n options \n" + 제시문2
                inputData = ansData["clue_text"] + "\n options \n" + str(ansData["options"])
                outputData = ansData["text"] # 정답
                rowData = [instrunction, inputData, outputData]
                alpaca_data.append(rowData)

        total_run_time = time.time() - start_time

        total_run_time = time.strftime("%H:%M:%S", time.gmtime(total_run_time))
        print(f"Run Time: {total_run_time}") # 총 실행 시간 출력

        return alpaca_data, column_list

    except Exception as e:
        print(str(e))

In [4]:
alpaca_data, column_list = load_training_data()

Run Time: 00:00:33


In [5]:
df = pd.DataFrame(data=alpaca_data, columns=column_list)
df

,instruction,input,output
0,센터장은 아동학대예방을 위해 누구에게 교육이 필요한지 위원들에게 전달했니,센터장의 설명이 끝난 후에는 센터장 및 강북구청 관계자와 특위 위원들 간의 질의 응...,부모
1,연구 성과 및 정보를 제공하는 보건환경톡톡의 7편의 내용은 무엇이야,"7편 하수찌꺼기 자원화, 8편 전통장류 품질 및 안전성, 9편 슈퍼푸드 잔류농약, ...",하수찌꺼기 자원화
2,무엇이 연구 실적 및 정보를 확인할 수 있는 보건환경톡톡의 7편의 내용일까,"7편 하수찌꺼기 자원화, 8편 전통장류 품질 및 안전성, 9편 슈퍼푸드 잔류농약, ...",하수찌꺼기 자원화
3,어떤 수단을 활용해 2021 제1차 강북구 치매안심센터 자문위원회 회의가 열릴 예정이니,건 명: 2021년 제1차 강북구 치매안심센터 자문위원회 서면개최\n options...,문서
4,한옥 등 건축자산의 진흥에 관한 법률 중 제8조에 해당하는 조례가 뭐야,❍ 서울특별시 각종 위원회의 설치·운영에 관한 조례 제8조(위원회의 구성)\n ...,서울특별시 각종 위원회의 설치·운영에 관한 조례
...,...,...,...
25161,"어떤 철새가 흑산도에서 11월 12월 두 달 동안 6,000마리 이상 발견됐지",하지만 11~12월 두 달 동안 어디에 머무는지 제대로 알려지지 않았는데 이번 조사...,괭이갈매기
25162,친환경 순환경제 정착을 위해 어떤 제도를 도입할 계획이야,"또한, 친환경 순환경제 정착을 위해 자원순환 성과관리제와 폐기물 처분분담금, 재활용...",재활용 네거티브제
25163,어떤 이행과제로 친환경 순환경제 정착을 촉진할 계획이지,"또한, 친환경 순환경제 정착을 위해 자원순환 성과관리제와 폐기물 처분분담금, 재활용...",재활용 네거티브제
25164,에너지 수요관리 강화를 위해 대한민국 정부가 추진하는 과제는 무엇이야,"(에너지 수요관리 강화) 수요관리형 전기요금제 확대, 에너지 수요관리 핵심기술 확보...",수요관리형 전기요금제 확대


In [6]:
# Dataset 형태로 변환
dataset = Dataset.from_pandas(df)

In [7]:
# 허깅페이스 로그인
huggingface_hub.login()

In [8]:
# 허깅페이스에 업로드
dataset.push_to_hub("woojin0412/common")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/woojin0412/common/commit/a4a95d805eb101d7bcf0bd73bf3547d3059b0929', commit_message='Upload dataset', commit_description='', oid='a4a95d805eb101d7bcf0bd73bf3547d3059b0929', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/woojin0412/common', endpoint='https://huggingface.co', repo_type='dataset', repo_id='woojin0412/common'), pr_revision=None, pr_num=None)

# 허깅페이스 데이터베이스 주소
https://huggingface.co/datasets/woojin0412/common